In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

sns.set_style("darkgrid")
sns.set_context("paper", font_scale=1.25)

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

color_palette = ["#fcbba1", "#fb6a4a", "#cb181d", "#67000d"]

In [ ]:
df = pd.read_csv("../../data/dataset.csv")

In [ ]:
melted_df = df.melt(
    value_vars=["reductionStatesUdBd", "reductionStatesUdBdp", "reductionTimeUdBd", "reductionTimeUdBdp"],
    var_name="Variable",
    value_name="Value",
)
melted_df["Category"] = melted_df["Variable"].map(
    {
        "reductionStatesUdBd": "Explored State Space",
        "reductionStatesUdBdp": "Explored State Space",
        "reductionTimeUdBd": "Execution Time",
        "reductionTimeUdBdp": "Execution Time",
    }
)
melted_df["Variant"] = melted_df["Variable"].map(
    {"reductionStatesUdBd": "BD", "reductionStatesUdBdp": "BDP", "reductionTimeUdBd": "BD", "reductionTimeUdBdp": "BDP"}
)

plt.figure(figsize=(4, 2.1))
ax = sns.boxplot(
    x="Category",
    y="Value",
    hue="Variant",
    data=melted_df,
    showfliers=False,
    showmeans=True,
    palette=color_palette,
    width=0.5,
    meanprops={"markerfacecolor": "black", "markeredgecolor": "black", "markersize": 6},
    linewidth=0.5,
)
plt.xlabel(None)
plt.ylabel("Reduction")
plt.legend(
    title="Variant",
    fontsize="x-small",
    title_fontsize="x-small",
    loc="upper left"
)

category_means = melted_df.groupby(["Category", "Variant"])["Value"].mean().reset_index()

for i in range(len(category_means)):
    category = category_means.loc[i, "Category"]
    method = category_means.loc[i, "Variant"]
    mean = category_means.loc[i, "Value"]
    x_pos = {"Explored State Space": 0, "Execution Time": 1}[category] + {"BD": -0.2, "BDP": 0.2}[method]
    ax.text(x_pos, mean + 0.1, f"{mean:.2f}", ha="center", va="bottom", color="black", fontsize=8)


category_quantiles = melted_df.groupby(["Category", "Variant"])["Value"].quantile([0.25, 0.5, 0.75]).unstack().reset_index()
category_quantiles.columns = ["Category", "Variant", "Q1", "Median", "Q3"]

categories = ax.get_xticks()

for cat in categories:
    category = ax.get_xticklabels()[cat].get_text()
    for variant, variant_name in enumerate(["BD", "BDP"]): 
        median_value = category_quantiles[(category_quantiles["Category"] == category) & (category_quantiles["Variant"] == variant_name)]["Median"].values[0]
        x_pos = cat - 0.12 if variant == 0 else cat + 0.12
        ax.text(
            x_pos, 
            median_value, 
            f'{median_value:.2f}', 
            ha='center', 
            va='center', 
            fontweight='bold', 
            size=5,
            color='black',
            bbox=dict(facecolor="lightgray", boxstyle="round,pad=0.3", edgecolor="lightgray")
        )

plt.tight_layout()
plt.show()

In [ ]:
df_copy = df.copy()

min_power = np.floor(np.log10(df_copy['bdVisitedStates'].min()))
max_power = np.ceil(np.log10(df_copy['bdVisitedStates'].max()))

bins = np.logspace(min_power, max_power, num=9)  

bin_labels = [f"$10^{int(np.log10(bins[i]))}$" for i in range(len(bins) - 1)]
df_copy['bin'] = pd.cut(df_copy['bdVisitedStates'], bins=bins, labels=bin_labels, include_lowest=True)


overall_mean = df_copy['reductionTimeBdBdp'].mean()

plt.figure(figsize=(5, 2))
ax = sns.lineplot(
    x='bin', 
    y='reductionTimeBdBdp',
    data=df_copy,
    marker="o",
    dashes=False,
    estimator='mean',
    color=color_palette[1],  
    errorbar=('se', 2)
)
ax.axhline(y=overall_mean, color='gray', linestyle='--', label=f'Overall Avg: {overall_mean:.2f}')

plt.legend(loc="lower right")  
plt.ylim(bottom=0, top=2)
plt.xlabel('Explored State Space Magnitude')
plt.ylabel('Reduction')
plt.show()

In [ ]:
filtered_df = df[['probSeq', 'probParr', 'probXor', 'probLoop', 'reductionStatesUdBd']].copy()
filtered_df[['probSeq', 'probParr', 'probXor', 'probLoop']] = filtered_df[['probSeq', 'probParr', 'probXor', 'probLoop']].round(1)

melted_df = filtered_df.melt(
    id_vars=["reductionStatesUdBd"],
    value_vars=["probSeq", "probParr", "probXor", "probLoop"],
    var_name="Type",
    value_name="Level"
)

melted_df = melted_df[melted_df['Level'] <= 0.8]

plt.figure(figsize=(5, 2.5))
ax = sns.lineplot(
    x="Level",
    y="reductionStatesUdBd",
    hue="Type",
    data=melted_df,
    estimator='mean',
    style="Type",
    markers='o',
    errorbar=("se", 2),
    dashes=False,
    palette=color_palette
)

plt.xlabel("Operator Level")
plt.ylabel("Reduction")
plt.grid(True, alpha=0.3)
ax.set_axisbelow(True)


handles, labels = ax.get_legend_handles_labels()
new_labels = ["Sequence", "Parallel", "Choice", "Loop"]
ax.legend(
    handles,
    new_labels,
    title="Operator",
    loc="upper left",
    fontsize="x-small",
    ncol=4,
    title_fontsize="x-small",
    handlelength=0.8,  
    handletextpad=0.2,
    handleheight=0.8 ,
)
plt.tight_layout()
ax.set_xticks(np.arange(0,0.9,0.1))
plt.show()